## Import librairies

In [121]:
import pandas as pd
import datetime

## Extract excel data in a pandas dataframe

In [287]:
db = pd.read_excel('datasource/database.xlsx', skiprows=2)

## Define the cleaning and transformation functions


### clean
Takes the db dataframe as argument and returns it after cleaning:
* Replace NaN values with empty string for object columns and 0 for float columns.
* Harmonise the gender column values
* Trim country column (remove leading and trailing spaces)
* Drop unused columns

In [194]:
def clean(db):

    db = db.apply(lambda col: col.fillna("") if col.dtype=="object" else col.fillna(0))
    db.gender.replace({"WOMEN'S": "WOMEN", "MEN'S": "MEN"}, inplace=True)
    db.country = db.country.str.strip()
    db.drop("product", axis=1, inplace=True)
    
    return (db)

### aggregate_by_segment
Takes the db dataframe and returns it after aggregating data at the segment level 

In [195]:
def groupby_segment(db):
    
    groups = ['brand', 'country', 'gender', 'product_line', 'segment']
    db = db.groupby(groups, as_index=False).sum()
    
    return (db)

### complete_product_line
Takes the db dataframe and rturns it after completing the 'product_line' column:
* Replace the 'product_line column by the concatenation of 'gender' and 'product_line' columns
* Trim the new 'product_line' column (to get rid of trailing spaces from empty 'gender' after concatenation)
* Drops the 'gender' column

In [196]:
def complete_product_line(db):
    
    db.product_line = db.gender + " " + db.product_line
    db.product_line = db.product_line.str.strip()
    db.drop('gender', axis=1, inplace=True)
    
    return (db)

### create_market_rows
Takes the db dataframe as argument and returns it with new rows for market data (sum of all brands)

In [197]:
def create_market_rows(db):
    
    groups = ['country', 'product_line', 'segment']
    tmp = db.groupby(groups, as_index=False).sum()
    tmp['brand'] = 'MARKET'
    db = db.append(tmp).reset_index(drop=True)
    
    return (db)

### reshape
Takes the db dataframe as argument and returns it after unpivoting date columns

In [198]:
def reshape(db):
    
    fixed_cols = db.select_dtypes(include=["object"]).columns
    db = db.melt(id_vars=fixed_cols, var_name='date')
    
    return (db)

### manage_dates
Takes the db dataframe as argument and returns it after creating date infromation fields:
* Splits the 'date' column into 'month' and 'year' columns
* Creates "month_no" column based on 'month'column (in short text format)

In [199]:
def manage_dates(db):
    
    db[['month', 'year']] = db.date.str.split(" ", expand=True)
    months = {datetime.datetime(2000, i, 1).strftime("%b"): i for i in range(1, 13)}
    db['month_no'] = db.month.map(months)
    
    return (db)

### compute_ytd
Takes the db dataframe as argument and returns it with a new 'ytd' column corresponding to the cumulated sum of 'value' across months for a given year:
* Sort (ascending order) the dataframe by, in order, 'brand', 'country', 'product_line', 'segment', 'year' and 'month_no'
* Compute the 'ytd' column thanks to the dedicated pandas 'cumsum()' function, combined to a 'groupby' on the relevant dimensions 

In [215]:
def compute_ytd(db):
    
    groups = ['brand', 'country', 'product_line', 'segment', 'year']
    sort_cols = groups + ['month_no']
    db = db.sort_values(sort_cols).reset_index(drop=True)
    db['ytd'] = db.groupby(groups)['value'].cumsum()
    
    return (db)

### compute_var_ytd
Takes the db dataframe as argument and returns it with a new 'var_ytd' column corresponding to the variation in % of the 'ytd' columns vs. the month of the previous year:
* Sort (ascending order) the dataframe by, in order, 'brand', 'country', 'product_line', 'segment', 'month_no' and 'year'
* Compute the 'var_ytd' column thanks to the dedicated pandas 'pct_change()' function, combined to a 'groupby' on the relevant dimensions 

In [224]:
def compute_var_ytd(db):
    
    groups = ['brand', 'country', 'product_line', 'segment', 'month_no']
    sort_cols = groups + ['year']
    db = db.sort_values(sort_cols).reset_index(drop=True)
    db['var_ytd'] = db.groupby(groups)['ytd'].pct_change()
    
    return (db)

### compute_market_share_ytd
Takes the db dataframe as argument and returns it with a new 'market_share_ytd' column corresponding to the % of the 'ytd' row value relative to the total of 'ytd' values for comparable dimensions:
* Create a 'total' dataframe that corresponds to the db dataframe filtered on 'MARKET' brand (i.e. the total of all brands)
* Make a left join of the db and 'total' dataframe on relevant dimensions
* Divide the db 'ytd' column by the'ytd_total' column to get the market share

In [ ]:
def compute_market_share_ytd(db):
    
    cols_for_join = ['country', 'product_line', 'segment', 'date']
    total = db.loc[db.brand == 'MARKET', cols_for_join + ['ytd']]
    db = db.merge(total, how='left', on=cols_for_join, suffixes=('', '_total'))
    db['market_share_ytd'] = db.ytd / db.ytd_total
    db.drop('ytd_total', axis=1, inplace=True)
    
    return (db)

## Apply cleaning and transformation functions to the dataframe

In [288]:
db = (db
      .pipe(clean)
      .pipe(groupby_segment)
      .pipe(complete_product_line)
      .pipe(create_market_rows)
      .pipe(reshape)
      .pipe(manage_dates)
      .pipe(compute_ytd)
      .pipe(compute_var_ytd)
      .pipe(compute_market_share_ytd)
     )

## TADA !!!

In [289]:
db

,brand,country,product_line,segment,date,value,month,year,month_no,ytd,var_ytd,market_share_ytd
0,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,Jan 2016,7262.801870,Jan,2016,1,7.262802e+03,NaN,0.841317
1,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,Jan 2017,8580.961640,Jan,2017,1,8.580962e+03,0.181495,0.527822
2,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,Feb 2016,6116.043680,Feb,2016,2,1.337885e+04,NaN,0.841317
3,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,Feb 2017,7226.072960,Feb,2017,2,1.580703e+04,0.181495,0.527822
4,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,Mar 2016,5733.790950,Mar,2016,3,1.911264e+04,NaN,0.841317
5,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,Mar 2017,6774.443400,Mar,2017,3,2.258148e+04,0.181495,0.527822
6,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,Apr 2016,5733.790950,Apr,2016,4,2.484643e+04,NaN,0.841317
7,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,Apr 2017,6774.443400,Apr,2017,4,2.935592e+04,0.181495,0.527822
8,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,May 2016,6498.296410,May,2016,5,3.134472e+04,NaN,0.841317
9,ANOTHER DAY IN PARADISE,CHINA,WOMEN PERFUME,BATH & BODY,May 2017,7677.702520,May,2017,5,3.703362e+04,0.181495,0.527822
